In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 18 02:34:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:

!pip -qqq install pydub
!pip -qqq install tensorboard
!pip -qqq install visualkeras
!pip -qqq install --upgrade keras
!pip -qqq install --upgrade keras tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.


In [5]:
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from pydub import AudioSegment
import random

import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix
from pydub import AudioSegment
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
base_dir = '/content/drive/MyDrive/ASE_Dataset_Folder'
#sound_dir = os.path.join(base_dir, 'Sound_1_Sec_5_Files')

image_dir = os.path.join(base_dir,  'STFT_Images_1_Sec')
train_dir = os.path.join(image_dir, 'Train/Sounds')
test_dir = os.path.join(image_dir, 'Test/Sounds')

# Check if the train_dir exists
if os.path.exists(image_dir):
  if os.path.exists(train_dir):
    if os.path.exists(test_dir):
      print(f"train_dir exists: {image_dir}")
  else:
    print(f"train_dir does not exist: {train_dir}")

#train_dir = r"/content/drive/MyDrive/ASE_Dataset_Folder/Images_1_Sec/Train"
#test_dir = r"/content/drive/MyDrive/ASE_Dataset_Folder/Images_1_Sec/Test"



# List files in the train directory
train_files = os.listdir(train_dir)
print(f"Files in train directory: {train_files[:10]}")  # Print first 10 files

# List files in the test directory
test_files = os.listdir(test_dir)
print(f"Files in test directory: {test_files[:10]}")  # Print first 10 files


train_dir exists: /content/drive/MyDrive/ASE_Dataset_Folder/STFT_Images_1_Sec
Files in train directory: ['Explosion', 'Emergency_alarm', 'Gun_Shots', 'glass_breaking', 'thunderstorm', 'dog', 'wind']
Files in test directory: ['Gun_Shots', 'Emergency_alarm', 'Explosion', 'dog', 'glass_breaking', 'thunderstorm', 'wind']


In [7]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 7
height = 128
n_fft = 2048
hop_length = 256
sr = 22050
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

# Output the results
print(f"Found {train_generator.samples} images for training.")
print(f"Found {test_generator.samples} images for testing.")

# Check if generators are yielding data
if len(train_generator) == 0:
    raise ValueError(f"Train generator is empty. Check the directory: {train_dir}")
if len(test_generator) == 0:
    raise ValueError(f"Test generator is empty. Check the directory: {test_dir}")


cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Flatten())

# RNN model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((width, height * channels), input_shape=(height, width, channels)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),
    tf.keras.layers.BatchNormalization()
])

# Combine CNN and RNN models
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((1, height, width, channels), input_shape=(height, width, channels)),
    tf.keras.layers.TimeDistributed(cnn_model),  # TimeDistributed applies the CNN model to each time step
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),  # RNN on top of CNN features
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'precision', 'recall', 'f1_score'])

epochs = 200
metrics_data = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1,
        validation_data=test_generator,
        validation_steps=len(test_generator))

    # Get metrics from history
    accuracy = history.history['accuracy'][0]
    precision = history.history['precision'][0]
    recall = history.history['recall'][0]

    f1_score_tensor = history.history['f1_score'][0]
    f1_score_value = np.mean(f1_score_tensor)

    # Store metrics in a list
    metrics_data.append([accuracy, precision, recall, f1_score_value])

model.save('/content/drive/MyDrive/CRNN/STFT_First_test/al-emadi_crnn_stft_standardized_unknown_1_test.keras')




Found 1111 images belonging to 7 classes.
Found 477 images belonging to 7 classes.
Found 1111 images for training.
Found 477 images for testing.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 907s 26s/step - accuracy: 0.2154 - f1_score: 0.2002 - loss: 1.9819 - precision: 0.4134 - recall: 0.0246 - val_accuracy: 0.1593 - val_f1_score: 0.0851 - val_loss: 4.1706 - val_precision: 0.1686 - val_recall: 0.1551
Epoch 2/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 23s 661ms/step - accuracy: 0.4158 - f1_score: 0.3812 - loss: 1.5253 - precision: 0.6969 - recall: 0.0888 - val_accuracy: 0.2075 - val_f1_score: 0.1426 - val_loss: 3.1350 - val_precision: 0.2990 - val_recall: 0.1279
Epoch 3/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 609ms/step - accuracy: 0.5008 - f1_score: 0.4470 - loss: 1.3895 - precision: 0.7170 - recall: 0.1720 - val_accuracy: 0.1614 - val_f1_score: 0.1124 - val_loss: 4.5775 - val_precision: 0.1656 - val_recall: 0.1614
Epoch 4/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 640ms/step - accuracy: 0.5186 - f1_score: 0.4689 - loss: 1.3173 - precision: 0.7147 - recall: 0.2513 - val_accuracy: 0.4046 - val_f1_score: 0.3529 - val_loss: 1.6046 - val_precision: 0.5172 - val_recall: 0.31

In [10]:
import csv

# Save metrics to a CSV file
csv_filename = '/content/drive/MyDrive/CRNN/STFT_First_test/training_metrics_1_test.csv'

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(metrics_data)

print(f"Training metrics saved to {csv_filename}")

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

Training metrics saved to /content/drive/MyDrive/CRNN/STFT_First_test/training_metrics_1_test.csv
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 347ms/step - accuracy: 0.9098 - f1_score: 0.5074 - loss: 0.3727 - precision: 0.9186 - recall: 0.9073
Test Loss: 0.5098955631256104
Test Accuracy: 0.8805031180381775
Test Precision: 0.8895965814590454
Test Recall: 0.8784067034721375
Test F1 Score: tf.Tensor(
[0.94488186 0.7953216  0.9275362  0.85436887 0.8870967  0.81132066
 0.9133858 ], shape=(7,), dtype=float32)


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Model and directories configuration
num_classes = 7
height = 128
n_fft = 2048
hop_length = 256
sr = 22050
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr
width = 1 + int((audio_length_samples - n_fft) / hop_length)
channels = 3
batch_size = 32

# Load the pre-trained model
model_path = '/content/drive/MyDrive/CRNN/STFT_First_test/al-emadi_crnn_stft_standardized_unknown_1_test.keras'
model = tf.keras.models.load_model(model_path)

# Test Data Generator
test_datagen = ImageDataGenerator()
test_dir = '/content/drive/MyDrive/ASE_Dataset_Folder/STFT_Images_1_Sec/Test/Sounds'  # Ensure this path is correct
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Evaluate the model
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
if len(results) > 2:
    print(f"Additional Metrics:")
    for metric, value in zip(model.metrics_names[2:], results[2:]):
        print(f"{metric.capitalize()}: {value}")

# Predicting classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# True class indices
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Classification Report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")


Found 477 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 387ms/step - accuracy: 0.9098 - f1_score: 0.5074 - loss: 0.3727 - precision: 0.9186 - recall: 0.9073
Test Loss: 0.5098955631256104
Test Accuracy: 0.8805031180381775
Additional Metrics:
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 388ms/step

Confusion Matrix:
[[120   4   0   0   0   0   0]
 [  4  68   0   0   4   1   1]
 [  0   1  32   1   0   1   0]
 [  4   0   0  44   5   3   0]
 [  1   1   2   1  55   0   0]
 [  0  12   0   1   0  43   2]
 [  1   7   0   0   0   0  58]]

Classification Report:
                 precision    recall  f1-score   support

Emergency_alarm       0.92      0.97      0.94       124
      Explosion       0.73      0.87      0.80        78
      Gun_Shots       0.94      0.91      0.93        35
            dog       0.94      0.79      0.85        56
 glass_breaking       0.86      0.92      0.89        60
   thunderstorm       0.90      0.74      0.81        58
           wind       0.95      0.88      0.91        66

       accuracy          